In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import requests
import json

In [2]:
#load in csv
olympics_data = pd.read_csv("Summer-Olympic-medals-1976-to-2008.csv", encoding="latin-1")
olympics_data.head()

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal
0,Montreal,1976.0,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver
1,Montreal,1976.0,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze
2,Montreal,1976.0,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold
3,Montreal,1976.0,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver
4,Montreal,1976.0,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze


In [3]:
# Assign Medal Counts
medal_dict = {'Gold':3,'Silver':2,'Bronze':1}
olympics_data['Points'] = olympics_data['Medal'].map(medal_dict)

In [4]:
# Load in Host Country CSV, merge with Olympics Data Frame
host_country_df = pd.read_csv('Host_Countries.csv')
host_country_df = host_country_df[['Year','Country']]
host_country_df.rename(columns={'Country':'Host Country'},inplace=True)
olympics_data = olympics_data.merge(host_country_df,on='Year')

In [5]:
# Get pt Total DF by year
pt_total_df = olympics_data.groupby('Year').sum()

In [6]:
# Find Host Years and where Host City Was

host_years = olympics_data['Year'].unique()

year_countries_dict = {}
for i in host_years:
    if i == 1988.0:
        year_countries_dict[i] = 'Korea, South'
    elif i == 1980.0:
        year_countries_dict[i] = 'Soviet Union'
    else:
        year_countries_dict[i] = olympics_data[olympics_data['Year']==i]['Host Country'].unique()[0]
        
# Create DF with only host countries

host_countries = []
for k,v in year_countries_dict.items():
    host_countries.append(v)

host_df = olympics_data[olympics_data['Country'].isin(host_countries)]

# Build Data Frame with a Year index and Host Country Point Totals as Columns

frame = pd.DataFrame(index=host_years)

for year in host_years:
    country = year_countries_dict[year]

    to_add = host_df[host_df['Country']==country].groupby('Year').sum()
    to_add.rename(columns={'Points':country},level=0,inplace=True)
    frame = frame.merge(to_add,how='outer',left_index=True,right_index=True)

frame.rename(columns={'United States_x':'United States'},inplace=True)
frame = frame[['Canada','Soviet Union','United States','Korea, South','Spain','Australia','Greece','China']]
frame.describe()

,Canada,Soviet Union,United States,"Korea, South",Spain,Australia,Greece,China
count,8.000000,3.000000,8.000000,8.000000,9.000000,9.000000,8.000000,7.000000
mean,70.875000,728.000000,553.875000,115.000000,74.111111,166.666667,18.000000,199.857143
std,46.208959,195.194775,147.069893,48.913919,59.258848,129.810439,21.553919,95.394669
min,28.000000,604.000000,337.000000,20.000000,9.000000,24.000000,1.000000,81.000000
25%,33.750000,615.500000,441.750000,101.750000,37.000000,76.000000,5.250000,153.000000
50%,56.000000,627.000000,572.500000,116.500000,50.000000,112.000000,8.500000,180.000000
75%,99.250000,790.000000,639.500000,143.750000,133.000000,253.000000,23.000000,223.500000
max,159.000000,953.000000,784.000000,171.000000,174.000000,372.000000,65.000000,385.000000


In [7]:
# Add Point Total COlumn
frame_pt = frame.merge(pt_total_df,left_index=True,right_index=True)

In [8]:
host_country_df['City'] = olympics_data['City'].unique()
host_country_df['Search'] = host_country_df['City']+', '+host_country_df['Host Country']

In [10]:
# Get Lat and Long

from config import g_key

city_dict = {}

search_items = []
for i in host_country_df['Search'].values:
    search_items.append(i)

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for item in search_items:
    
    params = {"address": item, "key": g_key}
    
    response = requests.get(base_url, params=params)

    geo = response.json()

    lat = geo["results"][0]["geometry"]["location"]["lat"]
    lng = geo["results"][0]["geometry"]["location"]["lng"]

    city_dict[item] = (lat,lng)

In [11]:
host_country_df['LAT'] = host_country_df['Search'].apply(lambda x: city_dict[x][0])
host_country_df['LNG'] = host_country_df['Search'].apply(lambda x: city_dict[x][1])

In [12]:
host_country_df

,Year,Host Country,City,Search,LAT,LNG
0,1976,Canada,Montreal,"Montreal, Canada",45.501689,-73.567256
1,1980,Russia,Moscow,"Moscow, Russia",55.755826,37.617300
2,1984,United States,Los Angeles,"Los Angeles, United States",34.052234,-118.243685
3,1988,South Korea,Seoul,"Seoul, South Korea",37.566535,126.977969
4,1992,Spain,Barcelona,"Barcelona, Spain",41.385064,2.173404
5,1996,United States,Atlanta,"Atlanta, United States",33.748995,-84.387982
6,2000,Australia,Sydney,"Sydney, Australia",-33.868820,151.209295
7,2004,Greece,Athens,"Athens, Greece",37.983810,23.727539
8,2008,China,Beijing,"Beijing, China",39.904200,116.407396


In [25]:
import gmaps

gmaps.configure(api_key=g_key)
locations = host_country_df[["LAT", "LNG"]]
points = frame_pt['Points'].values

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=mean_pts,
                                 dissipating=False, max_intensity=700,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
mean_pts = [70.875,728,553.875,115,553.875,74.11111,166.666667,18,199.857143]

In [21]:
frame.describe().T['mean']

Canada            70.875000
Soviet Union     728.000000
United States    553.875000
Korea, South     115.000000
Spain             74.111111
Australia        166.666667
Greece            18.000000
China            199.857143
Name: mean, dtype: float64

In [40]:
frame_pt[frame_pt.columns[:-1]]

,Canada,Soviet Union,United States,"Korea, South",Spain,Australia,Greece,China
1976.0,28.0,604.0,337.0,20.0,12.0,40.0,NaN,NaN
1980.0,NaN,953.0,NaN,NaN,47.0,24.0,7.0,NaN
1984.0,159.0,NaN,784.0,83.0,37.0,78.0,3.0,141.0
1988.0,34.0,627.0,411.0,170.0,9.0,76.0,1.0,81.0
1992.0,103.0,NaN,452.0,110.0,174.0,112.0,6.0,165.0
1996.0,98.0,NaN,628.0,135.0,136.0,212.0,20.0,222.0
2000.0,43.0,NaN,574.0,123.0,69.0,372.0,32.0,180.0
2004.0,33.0,NaN,571.0,108.0,50.0,333.0,65.0,225.0
2008.0,69.0,NaN,674.0,171.0,133.0,253.0,10.0,385.0
